### Hospital e Leito
Fonte: OpenDataSUS

In [44]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# De 01/24 a 07/24
hospital_e_leitos_br = pd.read_csv("dados/Leitos_2024.csv", encoding='ISO-8859-1')

# Limpeza de dados
hospital_e_leitos_pb = hospital_e_leitos_br[(hospital_e_leitos_br['MUNICIPIO'] == 'JOAO PESSOA') & (hospital_e_leitos_br['COMP'] <= 202407)]

hospital_e_leitos_pb = hospital_e_leitos_pb.drop(['REGIAO', 'UF', 'MOTIVO_DESABILITACAO', 'CO_TIPO_UNIDADE', 'NATUREZA_JURIDICA', 'NO_COMPLEMENTO'], axis=1)

hospital_e_leitos_pb[['NU_TELEFONE', 'NO_EMAIL']] = hospital_e_leitos_pb[['NU_TELEFONE', 'NO_EMAIL']].fillna('Não se aplica')

hospital_e_leitos_pb = hospital_e_leitos_pb.rename(columns={
    'COMP':'ANO_MES_COMPETENCIA',
    'CNES':'ID_CNES',
    'TP_GESTAO':'TIPO_GESTAO'
})

hospital_e_leitos_pb['TIPO_GESTAO'] = hospital_e_leitos_pb['TIPO_GESTAO'].replace({
    'M':'Municipal',
    'E':'Estadual',
    'D':'Dupla',
    'S':'Sem Gestão'
})

### SIH João Pessoa
Fonte: Base dos dados

In [45]:
# De 01/24 a 07/24
sih_jp_2024 = pd.read_csv('dados/sih_sus_joao_pessoa_2024.csv')

# Limpeza de dados
sih_jp_2024 = sih_jp_2024.drop(['ano', 'mes', 'id_municipio_paciente', 'id_gestor', 'id_aih', 'id_prestador_servico', 'cbo_2002_profissional', 'id_cid_secundario', 'id_cid_secundario_subcategoria', 'nota_fiscal', 'indicador_uf_hospital', 'indicador_uf_paciente', 'indicador_id_aih', 'tipo_financiamento_ato_profissional', 'tipo_subtipo_financiamento_ato_profissional', 'tipo_documento_pf', 'tipo_documento_pj', 'tipo_valor', 'sequencia', 'nome_remessa'], axis=1)

sih_jp_2024[['id_cid_principal', 'id_cid_principal_subcategoria']] = sih_jp_2024[['id_cid_principal', 'id_cid_principal_subcategoria']].fillna('Não se aplica')

C:\Users\Escritorio\AppData\Local\Temp\ipykernel_47892\3104296728.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  sih_jp_2024 = pd.read_csv('dados/sih_sus_joao_pessoa_2024.csv')


### DataFrame com o fluxo de internações e capacidade em leitos do SUS
Os dados são de janeiro até julho de 2024

In [ ]:
# transformando as datas em datetime
sih_jp_2024['data_entrada_internacao'] = pd.to_datetime(sih_jp_2024['data_entrada_internacao'])
sih_jp_2024['data_saida_iternacao'] = pd.to_datetime(sih_jp_2024['data_saida_iternacao'])
hospital_e_leitos_pb['mes_ano_leitos'] = pd.to_datetime(hospital_e_leitos_pb['ANO_MES_COMPETENCIA'].astype(str), format='%Y%m')
# pegando apenas o ano e mes e inserindo em nova coluna
sih_jp_2024['mes_ano'] = sih_jp_2024['data_entrada_internacao'].dt.to_period('M') 
sih_jp_2024['mes_ano_saida'] = sih_jp_2024['data_saida_iternacao'].dt.to_period('M')
hospital_e_leitos_pb['mes_ano_leitos'] = hospital_e_leitos_pb['mes_ano_leitos'].dt.to_period('M')

# pegando as quantidade de entrada, saidas e leitos por hospital e mês
qtd_entradas = sih_jp_2024.groupby(['id_estabelecimento_cnes', 'mes_ano']).size().reset_index(name='qtd_entradas')
qtd_saidas = sih_jp_2024.groupby(['id_estabelecimento_cnes', 'mes_ano_saida']).size().reset_index(name='qtd_saidas')
qtd_saidas = qtd_saidas.rename(columns={'mes_ano_saida': 'mes_ano'})
qtd_leitos_sus = hospital_e_leitos_pb[['ID_CNES', 'mes_ano_leitos', 'LEITOS_SUS']]

# juntando e limpando o novo df
df_ocupacao_mensal = pd.merge(qtd_entradas, qtd_saidas, on=['id_estabelecimento_cnes', 'mes_ano'], how='outer')
df_ocupacao_mensal = pd.merge(df_ocupacao_mensal, qtd_leitos_sus, left_on=['id_estabelecimento_cnes','mes_ano'], right_on=['ID_CNES', 'mes_ano_leitos'], how='left')
df_ocupacao_mensal = df_ocupacao_mensal.drop(['ID_CNES', 'mes_ano_leitos'], axis=1)
df_ocupacao_mensal = df_ocupacao_mensal.rename(columns= {'LEITOS_SUS':'total_leitos_sus'})
df_ocupacao_mensal = df_ocupacao_mensal[df_ocupacao_mensal['total_leitos_sus'].notna()]
df_ocupacao_mensal['total_leitos_sus'] = df_ocupacao_mensal['total_leitos_sus'].astype(int)
df_ocupacao_mensal['qtd_entradas'] = df_ocupacao_mensal['qtd_entradas'].fillna(0).astype(int)
df_ocupacao_mensal['qtd_saidas'] = df_ocupacao_mensal['qtd_saidas'].fillna(0).astype(int)

# criando a tabela de leitos_ocupados e taxa_ocupacao
df_ocupacao_mensal['leitos_ocupados'] = (df_ocupacao_mensal['qtd_entradas'] - df_ocupacao_mensal['qtd_saidas'])
df_ocupacao_mensal['taxa_ocupacao_percentual'] = (df_ocupacao_mensal['leitos_ocupados'] / df_ocupacao_mensal['total_leitos_sus']) * 100
df_ocupacao_mensal['taxa_ocupacao_percentual'] = df_ocupacao_mensal['taxa_ocupacao_percentual'].round(2)
df_ocupacao_mensal

,id_estabelecimento_cnes,mes_ano,qtd_entradas,qtd_saidas,total_leitos_sus,leitos_ocupados,taxa_ocupacao_percentual
0,147907,2024-01,2256,940,50,1316,2632.0
1,147907,2024-02,1651,1835,50,-184,-368.0
2,147907,2024-03,2245,2164,50,81,162.0
3,147907,2024-04,2137,2108,50,29,58.0
4,147907,2024-05,2211,2174,50,37,74.0
...,...,...,...,...,...,...,...
147,7870930,2024-03,155,163,10,-8,-80.0
148,7870930,2024-04,153,160,10,-7,-70.0
149,7870930,2024-05,133,100,10,33,330.0
150,7870930,2024-06,30,53,10,-23,-230.0
